In [1]:
#Run this cell to move the files to /shared-data - only need to run for the first time
!mv aisdk_20181102.csv /shared-data/ais_data/aisdk_20181102.csv

In [2]:
import sys
sys.path.insert(0, '/usr/local/spark/jars/samples.zip')
sys.path.insert(0, '/usr/local/spark/jars/xgboost4j-spark_3.0-1.0.0-0.2.0.jar')
sys.path.insert(0, '/usr/local/spark/jars/xgboost4j-1.0.0-0.2.0.jar')

In [3]:
from pyspark.sql import SparkSession
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.sql import SparkSession
from pyspark.sql.types import FloatType, IntegerType, StructField, StructType
from time import time

spark = SparkSession.builder \
    .config('spark.executor.cores','1')\
    .config('driver-memory', '10G')\
    .getOrCreate()

In [4]:
spark

In [5]:
df = spark.read.option("header",True) \
     .csv('/shared-data/ais_data/*.csv')

In [6]:
df.show(20)

+-------------------+--------------+---------+---------+----------+--------------------+----+----+-----+-------+-------+--------+------------------+---------+----------+-----+------+------------------------------+-------+-----------+----+----------------+----+----+----+----+
|        # Timestamp|Type of mobile|     MMSI| Latitude| Longitude| Navigational status| ROT| SOG|  COG|Heading|    IMO|Callsign|              Name|Ship type|Cargo type|Width|Length|Type of position fixing device|Draught|Destination| ETA|Data source type|   A|   B|   C|   D|
+-------------------+--------------+---------+---------+----------+--------------------+----+----+-----+-------+-------+--------+------------------+---------+----------+-----+------+------------------------------+-------+-----------+----+----------------+----+----+----+----+
|02/11/2018 00:00:00|       Class A|219001182|55.822560| 10.063892|Under way using e...| 0.0| 0.0|272.0|    272|Unknown|    null|              null|Undefined|      null| nu

In [7]:
MMSI_list = ['212963000','219000751','219015591','305837000','311646000']
df=df.filter(df.MMSI.isin(MMSI_list)).withColumnRenamed('# Timestamp','Timestamp')

In [8]:
df_plot=df.filter(df.MMSI.isin(MMSI_list[0])).select('Latitude','Longitude')
df_plot.show(10)

+---------+---------+
| Latitude|Longitude|
+---------+---------+
|58.273420|10.959500|
|58.273038|10.959695|
|58.272593|10.959918|
|58.272277|10.960082|
|58.271895|10.960285|
|58.271450|10.960507|
|58.271133|10.960670|
|58.270690|10.960890|
|58.270312|10.961085|
|58.269995|10.961245|
+---------+---------+
only showing top 10 rows



In [9]:
df_simplified=df[['Timestamp','MMSI','Latitude','Longitude']]

In [10]:
df_simplified.show(20)

+-------------------+---------+---------+---------+
|          Timestamp|     MMSI| Latitude|Longitude|
+-------------------+---------+---------+---------+
|02/11/2018 00:00:00|212963000|58.273420|10.959500|
|02/11/2018 00:00:00|311646000|58.046110|10.192865|
|02/11/2018 00:00:00|219000751|55.248292|14.836152|
|02/11/2018 00:00:01|305837000|54.597698|12.253178|
|02/11/2018 00:00:03|305837000|54.597563|12.253113|
|02/11/2018 00:00:05|305837000|54.597430|12.253045|
|02/11/2018 00:00:05|212963000|58.273038|10.959695|
|02/11/2018 00:00:06|311646000|58.045953|10.192062|
|02/11/2018 00:00:07|305837000|54.597230|12.252943|
|02/11/2018 00:00:08|219015591|57.213833| 9.686700|
|02/11/2018 00:00:09|305837000|54.597163|12.252910|
|02/11/2018 00:00:10|219000751|55.248307|14.836150|
|02/11/2018 00:00:11|305837000|54.597028|12.252842|
|02/11/2018 00:00:12|212963000|58.272593|10.959918|
|02/11/2018 00:00:13|305837000|54.596895|12.252775|
|02/11/2018 00:00:14|219000751|55.248307|14.836150|
|02/11/2018 

In [11]:
df_simplified.write.mode('overwrite').option("header", "true").csv('spark_results.csv')

In [12]:
spark.stop()